# 在 Google Colab 上產圖：PyGMT 與示範

這個 notebook 是為了在 Google Colab（或任何有 sudo 權限的雲端 runtime）上執行 PyGMT 產生地圖圖片而準備。內容包含：

1. 設定/說明 Colab runtime 與如何掛載 Google Drive
2. 安裝系統依賴（GMT / GDAL / netCDF 等）與 Python 套件 (PyGMT)
3. 三個 PyGMT 範例：地形、地震疊加、球面投影 + 熱點 → 輸出 PNG
4. 顯示、檢查與下載產生的 PNG 檔案

使用順序：請按 cell 順序由上而下執行（或依需求選擇要執行的段落）。

## 1) Colab 執行環境與權限設定

這一段示範如何檢查硬體加速（GPU/TPU）以及如何把 Colab 和 Google Drive 串接（選項）。

請注意：在 Colab 中執行下列 `drive.mount()` 會要求你授權 Google Drive 存取權限。


In [ ]:
# 檢查硬體加速（如果在 Colab 執行）
import os
try:
    # GPU
    gpu = os.environ.get('COLAB_GPU') or os.popen('nvidia-smi -L').read().strip()
    print('GPU info:', gpu if gpu else 'no GPU found')
except Exception:
    print('無法檢查 GPU（可能不是 Colab 環境）')

# 在 Colab 中，如果需要把檔案存到 Google Drive，請解除下面註解並執行
# from google.colab import drive
# drive.mount('/content/drive')


In [ ]:
%%bash
# 安裝系統依賴（可能需要幾分鐘）
# Note: 在 Colab 會有 sudo 權限
apt-get update -qq
apt-get install -y -qq gmt gdal-bin libnetcdf-dev libhdf5-dev proj-bin libproj-dev

# 安裝 PyGMT 與常用圖表套件
python3 -m pip install --upgrade pip setuptools wheel
python3 -m pip install --no-cache-dir pygmt matplotlib seaborn plotly pillow opencv-python ipywidgets

# 確認版本
python3 -c "import pygmt, matplotlib, seaborn, plotly; print('pygmt', pygmt.__version__, 'matplotlib', matplotlib.__version__, 'seaborn', seaborn.__version__)"


In [ ]:
# 匯入常用 Python 繪圖套件與設定
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from PIL import Image

%matplotlib inline
sns.set_style('darkgrid')
print('環境載入完成')


## 3) 載入資料（上傳 / Google Drive / 網路）

# 1) 直接上傳本機檔案（Colab）
# from google.colab import files
# uploaded = files.upload()
# uploaded.keys()

# 2) 從 Google Drive 讀取（需先 drive.mount）
# import pandas as pd
# df = pd.read_csv('/content/drive/MyDrive/somefile.csv')

# 3) 從網路下載
# import requests
# r = requests.get('https://example.com/data.csv')
# open('data.csv','wb').write(r.content)

print('示範：上傳 / Drive / 網路 三種載入方式，請在 Colab 中依需求執行對應 cell。')

In [ ]:
## 4) Matplotlib 範例 — 折線 / 散佈 / 直方圖

x = np.linspace(0, 10, 200)
y = np.sin(x) + np.random.normal(0, 0.12, x.size)

plt.figure(figsize=(8,3))
plt.plot(x, y, label='sin + noise')
plt.title('簡單折線圖')
plt.legend()
plt.show()

plt.figure(figsize=(6,4))
plt.scatter(x, y, alpha=0.6, s=18)
plt.title('散佈圖')
plt.show()

plt.figure(figsize=(6,3))
plt.hist(y, bins=25)
plt.title('直方圖')
plt.show()

In [ ]:
## 5) Seaborn 範例 — box / violin / pairplot / heatmap

import seaborn as sns
iris = sns.load_dataset('iris')
sns.pairplot(iris, hue='species')
plt.show()

plt.figure(figsize=(6,3))
sns.boxplot(x='species', y='sepal_length', data=iris)
plt.title('Boxplot 範例')
plt.show()

plt.figure(figsize=(6,3))
sns.heatmap(iris.corr(), annot=True, fmt='.2f', cmap='vlag')
plt.title('Correlation heatmap')
plt.show()


In [ ]:
## 6) Plotly 互動式範例（離線模式）
import plotly.express as px
fig = px.scatter(iris, x='sepal_length', y='sepal_width', color='species', size='petal_length')
fig.update_layout(title='Iris Plotly 散佈示例')
fig.show()


In [ ]:
## 7) 影像讀取與顯示（PIL / OpenCV 範例）
from PIL import Image
import cv2

# 範例：用 Pillow 產生一張測試圖（若沒有上傳）
img = Image.new('RGB', (640,360), color=(64,128,192))
draw = Image.ImageDraw if hasattr(Image, 'ImageDraw') else None
img.save('test_pil_example.png')

# 使用 OpenCV 讀入並顯示（OpenCV 使用 BGR）
im_cv = cv2.imread('test_pil_example.png')
if im_cv is not None:
    im_rgb = cv2.cvtColor(im_cv, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(6,3))
    plt.imshow(im_rgb)
    plt.axis('off')
    plt.title('PIL 產生的測試影像（OpenCV 讀回顯示）')
    plt.show()
else:
    print('OpenCV 讀圖失敗')


In [ ]:
## 8) 圖表美化與自訂（標題、標籤、圖例、色彩映射）
plt.figure(figsize=(7,3))
plt.scatter(x, y, c=x, cmap='viridis', s=30)
plt.colorbar(label='value')
plt.title('色彩映射示例')
plt.xlabel('x')
plt.ylabel('y')
plt.show()


In [ ]:
## 9) 互動式控制：ipywidgets 範例
from ipywidgets import interact, IntSlider

@interact(n=IntSlider(min=5,max=100, step=5, value=25))
def draw_hist(n=25):
    data = np.random.randn(1000) * (n/25)
    plt.figure(figsize=(6,3))
    plt.hist(data, bins=30)
    plt.title(f'Histogram with scale {n}');
    plt.show()


In [ ]:
## 10) 批次產圖（PyGMT）：非洲地形 / 地震疊加 / 球面投影 + 熱點
# 這個 cell 會嘗試產生三張圖片：africa_topography.png, africa_quakes.png, africa_globe_hotspots.png

import pygmt
from pygmt import Figure
from pygmt.datasets import load_earth_relief

out_dir = '/content/pygmt_outputs'
import os
os.makedirs(out_dir, exist_ok=True)

print('PyGMT version', pygmt.__version__)
region = '-30/60/-40/40'  # lon_min/lon_max/lat_min/lat_max

# helper to load earth relief (try higher-res then fallback)
def load_relief_prefer(res_list=['02m','05m','01d'], region=region):
    for res in res_list:
        try:
            print('Trying resolution', res)
            g = load_earth_relief(resolution=res, region=region)
            print('Loaded relief at', res)
            return g
        except Exception as e:
            print('Failed to load', res, '->', e)
    raise RuntimeError('Could not load any earth_relief for requested region')

# 1) Topography
try:
    grid = load_relief_prefer(['02m','05m','01d'])
    fig = Figure()
    fig.basemap(region=region, projection='M18c', frame=True)
    fig.grdimage(grid=grid, cmap='geo', shading=True)
    fig.coast(resolution='f', shorelines=True)
    fig.colorbar(frame='af+l"Elevation (m)"')
    out1 = os.path.join(out_dir, 'africa_topography.png')
    fig.savefig(out1)
    print('Saved', out1)
except Exception as e:
    print('Topography generate failed:', e)

# 2) Quakes overlay
try:
    grid2 = load_relief_prefer(['05m','01d'])
    fig = Figure()
    fig.basemap(region=region, projection='M18c', frame=['WSne','af'])
    fig.grdimage(grid=grid2, cmap='geo', shading=True)
    # try to load builtin quakes dataset
    try:
        quakes = pygmt.datasets.load_earthquakes()
        print('Quakes dataset length:', len(quakes))
        fig.plot(x=quakes.longitude, y=quakes.latitude, style='cc', color='red', size='0.08c')
    except Exception as qe:
        print('Could not load quakes dataset, plotting fallback points:', qe)
        fig.plot(x=[20,35,-10], y=[0,-20,10], style='cc', color='red', size='0.1c')
    fig.coast(shorelines=True, resolution='f')
    out2 = os.path.join(out_dir, 'africa_quakes.png')
    fig.savefig(out2)
    print('Saved', out2)
except Exception as e:
    print('Quakes generate failed:', e)

# 3) Globe / hotspots
try:
    grid3 = load_relief_prefer(['05m','01d'])
    fig = Figure()
    fig.basemap(region='g', projection='G20/0/12c', frame=True)
    fig.grdimage(grid=grid3, cmap='geo', shading=True)
    try:
        hotspots = pygmt.datasets.load_hotspots()
        print('Hotspots length:', len(hotspots))
        fig.plot(x=hotspots.longitude, y=hotspots.latitude, style='c0.15c', color='yellow')
    except Exception as he:
        print('Hotspots dataset missing, drawing fallback points', he)
        fig.plot(x=[20,10,30], y=[0,-10,12], style='c0.15c', color='yellow')
    out3 = os.path.join(out_dir, 'africa_globe_hotspots.png')
    fig.savefig(out3)
    print('Saved', out3)
except Exception as e:
    print('Globe generate failed:', e)

print('\nAll generation attempts finished; check', out_dir)


In [ ]:
## 11) 儲存 / 上傳 / 下載說明（Drive 或直接下載）
from IPython.display import Image, display

out_dir = '/content/pygmt_outputs'
files = ['africa_topography.png', 'africa_quakes.png', 'africa_globe_hotspots.png']

print('列出輸出檔')
for f in files:
    fp = os.path.join(out_dir, f)
    if os.path.exists(fp):
        print('-', fp, 'size:', os.path.getsize(fp))
        display(Image(fp))
    else:
        print('-', fp, 'NOT FOUND')

# 如要把結果存到 Google Drive（先 mount）
# drive_dest = '/content/drive/MyDrive/pygmt_outputs'
# os.makedirs(drive_dest, exist_ok=True)
# for f in files:
#     src = os.path.join(out_dir, f)
#     if os.path.exists(src):
#         shutil.copy(src, drive_dest)
# print('Copied to Drive at', drive_dest)

# 下載檔案（Colab -> 本地）請使用：
# from google.colab import files; files.download('/content/pygmt_outputs/africa_topography.png')


In [ ]:
## 12) 單元測試與輸出檢查
# 簡單 assert 測試：檔案存在且大小 > 1 KB
ok = True
for f in files:
    fp = os.path.join(out_dir, f)
    if not os.path.exists(fp):
        print('MISSING', fp)
        ok = False
    elif os.path.getsize(fp) < 1024:
        print('TOO SMALL', fp, os.path.getsize(fp))
        ok = False

assert ok, '有檔案缺失或尺寸過小，請檢查上面的輸出與錯誤訊息'
print('所有檔案檢查通過 ✔')
